**EP3 - MAP2212 - 2024**

Victor Rocha Cardoso Cruz,                   NUSP: 11223757

Larissa Aparecida Marques Pimenta Santos,    NUSP: 12558620


    ENUNCIADO:
    Neste EP, você deve analisar experimentalmente a velocidade de convergência dos procedimentos de integração implementados no EP 2 ao substituir o gerador
    de números pseudo-aleatórios por um gerador quase-aleatório.
      - Como um aprimoramento do EP 2, considere trocar o gerador de números pseudo-aleatórios por um gerador de números quase-aleatórios.
      - Suas rotinas de integração por Monte Carlo funcionam melhor? Empiricamente, o quão mais rápidas são as novas rotinas?
      - Explique cuidadosamente por que e como você fez suas análises empíricas e chegou à sua conclusão.

In [ ]:
'''

CÓDIGO DO EP 2 com contagem de tempo

'''
import numpy as np
from scipy.stats import uniform, beta, gamma, weibull_min
import time

# Definição dos parâmetros a e b
a = 0.564263989  # RG
b = 0.51197762809  # CPF

# Função alvo f(x)
def f(x):
    fx = np.exp(-a * x) * np.cos(b * x)
    return fx

print("*** EP 2 INICIANDO ***")
print()
print("*** EXPERIMENTO PILOTO ***")
# EXPERIMENTO PILOTO #
# Método Cru para definição da integral esperada
# Definição a largura da distribuição
n = 1000000
uniform_samples_x = uniform.rvs(size=n, loc=0, scale=1)
uniform_samples_y = f(uniform_samples_x)
# Aproximação através de Distribuição Uniforme
AreaPilot = np.mean(uniform_samples_y)
print(f"Área Piloto {AreaPilot}")
print()

#Definição da margem de erro e quantil da normal
epsilon = 0.05/100 * AreaPilot
z_alfa = 1.64

total_time = 0

# ********************* MÉTODO DE MONTE CARLO CRU ********************* #
print("*** MÉTODO DE MONTE CARLO CRU ***")
# Distribuição Uniforme
n = 100000
start_time = time.time()
uniform_samples_x = uniform.rvs(size=n, loc=0, scale=1)
uniform_samples_y = f(uniform_samples_x)
# Aproximação através de Distribuição Uniforme
area_cru = np.mean(uniform_samples_y)
variancia = np.sum((uniform_samples_y - area_cru)**2)/n
erro_padrao = np.sqrt(variancia)/np.sqrt(n)
end_time = time.time()
execution_time = end_time - start_time
total_time += execution_time
print(f"Área estimada: {area_cru}")
print(f"Variância do estimador: {variancia}")
print(f"Erro padrão do estimador: {erro_padrao:.10f}")
print(f"Largura da Distr.: {n}")
print()
n = round(((z_alfa * np.sqrt(variancia))/epsilon)**2)
print(f"Valor de n calculado: {n}")
print("Tempo de execução:", execution_time, "segundos")
print()

# ******************************************************************************* #
# ********************* MÉTODO DE MONTE CARLO 'HIT OR MISS' ********************* #
print("*** MÉTODO DE MONTE CARLO 'HIT OR MISS' ***")
# Aproximação através de Distribuição Uniforme
n = 100000
PontosDentro = 0
start_time = time.time()
for i in range(n):
    xi = uniform.rvs()
    yi = uniform.rvs()
    # Checa se o ponto sorteado está abaixo da curva
    if yi <= f(xi):
        PontosDentro += 1
area_hitmiss = PontosDentro/n
# Variancia do estimador
variancia = area_hitmiss*(1 - area_hitmiss)/n
erro_padrao = np.sqrt(variancia)/np.sqrt(n)
end_time = time.time()
execution_time = end_time - start_time
total_time += execution_time
print(f"Área estimada: {area_hitmiss}")
print(f"Variância do estimador: {variancia:.10f}")
print(f"Erro padrão do estimador: {erro_padrao:.10f}")
print(f"Largura da Distr.: {n}")
print()
n = round(((z_alfa * np.sqrt(variancia))/epsilon)**2)
print(f"Valor de n calculado: {n}")
print("Tempo de execução:", execution_time, "segundos")
print()

# ******************************************************************************************** #
# ********************* MÉTODO DE MONTE CARLO AMOSTRAGEM POR IMPORTÂNCIA ********************* #
print("*** MÉTODO DE MONTE CARLO AMOSTRAGEM POR IMPORTÂNCIA ***")
# Distribuição Uniforme
n = 100000
pesos = []
pdfx = []
start_time = time.time()
xi = uniform.rvs(size=n, loc=0, scale=1)
# Função Densidade de probabilidade da Distribuição Uniforme
pdfx = uniform.pdf(xi)
yi = f(xi)
# Calculo dos pesos, como o quociente entre os valores f(xi) da função pela FDP da distribuição uniforme
pesos = yi/pdfx
area_uniform = np.mean(np.array(pesos))
# Variancia do estimador
variancia = np.sum(np.array(pdfx)*(np.array(pesos) - area_uniform)**2)/n
erro_padrao = np.sqrt(variancia)/np.sqrt(n)
end_time = time.time()
execution_time = end_time - start_time
total_time += execution_time
print(f"Uniforme: Área estimada: {area_uniform}")
print(f"Uniforme: Variância do estimador: {variancia:.10f}")
print(f"Uniforme: Erro padrão do estimador: {erro_padrao:.10f}")
print(f"Uniforme: Quantidade de iterações: {n}")
print()
n = round(((z_alfa * np.sqrt(variancia))/epsilon)**2)
print(f"Valor de n calculado: {n}")
print("Tempo de execução:", execution_time, "segundos")
print()

# Distribuição Beta
# Calculo dos pesos, como o quociente entre os valores f(xi) da função pela FDP da distribuição Beta
pesos = []
pdfx = []
n = 100000
start_time = time.time()
xi = beta.rvs(size=n, a = 1, b = 1)
# Obtenção da função densidade de probabilidade, para a distribuição Beta
pdfx = beta.pdf(xi, a = 1, b = 1)
yi = f(xi)
pesos = yi/pdfx
area_beta = np.mean(pesos)
# Variancia do estimador
variancia = np.sum(np.array(pdfx)*(np.array(pesos) - area_beta)**2)/n
erro_padrao = np.sqrt(variancia)/np.sqrt(n)
end_time = time.time()
execution_time = end_time - start_time
total_time += execution_time
print(f"Beta: Área estimada: {area_beta}")
print(f"Beta: Variância do estimador: {variancia:.10f}")
print(f"Beta: Erro padrão do estimador: {erro_padrao:.10f}")
print(f"Beta: Quantidade de iterações: {n}")
print()
n = round(((z_alfa * np.sqrt(variancia))/epsilon)**2)
print(f"Valor de n calculado: {n}")
print("Tempo de execução:", execution_time, "segundos")
print()

# Distribuição Gamma
# Normalização da distribuição Gamma a partir da função cumulativa nos pontos de interesse
cdf_lower = gamma.cdf(0, a=1, scale=2)
cdf_upper = gamma.cdf(1, a=1, scale=2)
const_normalizacao = 1 / (cdf_upper - cdf_lower)
# Função criada para obter apenas valores entre 0 e 1
def generate_n_gamma_samples(num_samples):
    samples_no_intervalo = []
    while len(samples_no_intervalo) < num_samples:
        # Gerar amostras aleatórias pela distribuição Gama
        gamma_samples_x = gamma.rvs(a=1, scale=2)
        # Checa se a amostra está dentro do intervalo
        if 0 <= gamma_samples_x <= 1:
            samples_no_intervalo.append(gamma_samples_x)
    return np.array(samples_no_intervalo)
pesos = []
pdfx = []
n = 100000
start_time = time.time()
xi = generate_n_gamma_samples(n)
# Obtenção da função densidade de probabilidade, para a distribuição Gamma, e normalizada à 0 e 1
pdf_value = gamma.pdf(xi, a = 1, scale = 2)
pdfx = pdf_value * const_normalizacao
yi = f(xi)
pesos = yi/pdfx
area_gama = np.mean(pesos)
# Variancia do estimador
variancia = np.sum(np.array(pdfx)*(np.array(pesos) - area_gama)**2)/n
erro_padrao = np.sqrt(variancia)/np.sqrt(n)
end_time = time.time()
execution_time = end_time - start_time
total_time += execution_time
print(f"Gama: Área estimada: {area_gama}")
print(f"Gama: Variância do estimador: {variancia}")
print(f"Gama: Erro padrão do estimador: {erro_padrao:.10f}")
print(f"Gama: Quantidade de iterações: {n}")
print()
n = round(((z_alfa * np.sqrt(variancia))/epsilon)**2)
print(f"Valor de n calculado: {n}")
print("Tempo de execução:", execution_time, "segundos")
print()

# Distribuição Weibull
# Normalização da distribuição Weibull a partir da função cumulativa nos pontos de interesse
cdf_lower = weibull_min.cdf(0, c=1)
cdf_upper = weibull_min.cdf(1, c=1)
const_normalizacao = 1 / (cdf_upper - cdf_lower)
# Função criada para obter apenas valores entre 0 e 1
def generate_n_weibull_samples(num_samples):
    samples_no_intervalo = []
    while len(samples_no_intervalo) < num_samples:
        # Gerar amostras aleatórias pela distribuição Weibull
        weibull_samples_x = weibull_min.rvs(c=1)
        # Checa se a amostra está dentro do intervalo
        if 0 <= weibull_samples_x <= 1:
            samples_no_intervalo.append(weibull_samples_x)
    return np.array(samples_no_intervalo)
pesos = []
pdfx = []
n = 100000
start_time = time.time()
xi = generate_n_weibull_samples(n)
# Obtenção da função densidade de probabilidade, para a distribuição Weibull, e normalizada à 0 e 1
pdf_value = weibull_min.pdf(xi, c=1)
pdfx = pdf_value * const_normalizacao
yi = f(xi)
pesos = yi/pdfx
area_weibull = np.mean(pesos)
# Variancia do estimador
variancia = np.sum(np.array(pdfx)*(np.array(pesos) - area_weibull)**2)/n
erro_padrao = np.sqrt(variancia)/np.sqrt(n)
end_time = time.time()
execution_time = end_time - start_time
total_time += execution_time
print(f"Weibull: Área estimada: {area_weibull}")
print(f"Weibull: Variância do estimador: {variancia:.10f}")
print(f"Weibull: Erro padrão do estimador: {erro_padrao:.10f}")
print(f"Weibull: Quantidade de iterações: {n}")
print()
n = round(((z_alfa * np.sqrt(variancia))/epsilon)**2)
print(f"Valor de n calculado: {n}")
print("Tempo de execução:", execution_time, "segundos")
print()

# *************************************************************************************** #
# ********************* MÉTODO DE MONTE CARLO VARIÁVEIS DE CONTROLE ********************* #
print("*** MÉTODO DE MONTE CARLO VARIÁVEIS DE CONTROLE ***")
# Definição dos limites de integração para a função f e φ
limInfer, limSuper= 0, 1
# Função polinominal φ que aproxima-se à curva da função f(x), encontrada a partir dos pontos (1, f(1)) e (0, f(0))
def phi(x):
    phix = ((f(1)-f(0))/(1 - 0))*x + 1
    #phix = 1 - 0.564263989*x + 0.02814*x**(2) + 0.04400*x**(3)- 0.01377*x**(4)
    return phix
# Função primitiva de φ
def phiPrim(x):
    phiPrim = x + ((f(1)-f(0))/(1 - 0))*x**(2)/2
    #phiPrim = x - 0.564263989*x**(2)/2 + 0.02814*x**(3)/3 + 0.04400*x**(4)/4 - 0.01377*x**(5)/5
    return phiPrim

termo = 0
fxn, phixn = [], []
n = 100000
start_time = time.time()
xi = uniform.rvs(size=n, loc=0, scale=1)
    # Armezando os valores de f(xi) e g(xi) para o cálculo final da variância
fxn = f(xi)
phixn = phi(xi)
# Termo do somatório para calculo de gama chapeu
termo = np.sum(f(xi) - phi(xi) + (phiPrim(limSuper)-phiPrim(limInfer)))
area_control = termo/n
var_f = np.var(np.array(fxn))
var_phi = np.var(np.array(phixn))
correlacao = np.cov(fxn, phixn)[0,1]
variancia = (1/n)*(var_f + var_phi - 2*correlacao*np.sqrt(var_f)*np.sqrt(var_phi))
erro_padrao = np.sqrt(variancia)/np.sqrt(n)
end_time = time.time()
execution_time = end_time - start_time
total_time += execution_time
print(f"Área estimada:: {area_control}")
print(f"Variância do estimador: {variancia:.10f}")
print(f"Erro padrão do estimador: {erro_padrao:.10f}")
print(f"Quantidade de iterações: {n}")
print()
n = round(((z_alfa * np.sqrt(variancia))/epsilon)**2)
print(f"Valor de n calculado: {n}")
print("Tempo de execução:", execution_time, "segundos")
print()
print("Tempo total de execução:", total_time, "segundos")


*** EP 2 INICIANDO ***

*** EXPERIMENTO PILOTO ***
Área Piloto 0.7355752609427533

*** MÉTODO DE MONTE CARLO CRU ***
Área estimada: 0.7353771765168675
Variância do estimador: 0.02142611121801039
Erro padrão do estimador: 0.0004628835
Largura da Distr.: 100000

Valor de n calculado: 426027
Tempo de execução: 0.0047986507415771484 segundos

*** MÉTODO DE MONTE CARLO 'HIT OR MISS' ***
Área estimada: 0.73635
Variância do estimador: 0.0000019414
Erro padrão do estimador: 0.0000044061
Largura da Distr.: 100000

Valor de n calculado: 39
Tempo de execução: 10.617778539657593 segundos

*** MÉTODO DE MONTE CARLO AMOSTRAGEM POR IMPORTÂNCIA ***
Uniforme: Área estimada: 0.7358535665223241
Uniforme: Variância do estimador: 0.0213930035
Uniforme: Erro padrão do estimador: 0.0004625257
Uniforme: Quantidade de iterações: 100000

Valor de n calculado: 425368
Tempo de execução: 0.008132696151733398 segundos

Beta: Área estimada: 0.7355480569465704
Beta: Variância do estimador: 0.0213651746
Beta: Erro pad

In [ ]:
'''

CÓDIGO DO EP 3

'''
import numpy as np
from scipy.stats import uniform, beta, gamma, weibull_min, qmc
import time

# Definição dos parâmetros a e b
a = 0.564263989  # RG
b = 0.51197762809  # CPF

# Função alvo f(x)
def f(x):
    fx = np.exp(-a * x) * np.cos(b * x)
    return fx

print("*** EP 3 INICIANDO ***")
print()
print("*** EXPERIMENTO PILOTO ***")
# EXPERIMENTO PILOTO #
# Método Cru para definição da integral esperada
# Definição a largura da distribuição
n = 1000000
# Amostragem usando o método de Halton
halton_samples_x = qmc.Halton(d=1).random(n)
halton_samples_y = f(halton_samples_x)
# Aproximação através de Distribuição Halton
AreaPilot = np.mean(halton_samples_y)
print(f"Área Piloto {AreaPilot}")
print()

# Definição da margem de erro e quantil da normal
epsilon = 0.05 / 100 * AreaPilot
z_alfa = 1.64

total_time = 0

# ********************* MÉTODO DE MONTE CARLO CRU ********************* #
print("*** MÉTODO DE MONTE CARLO CRU ***")
# Distribuição Uniforme
n = 100000
start_time = time.time()
halton_samples_x = qmc.Halton(d=1).random(n)
halton_samples_y = f(halton_samples_x)
# Aproximação através de Distribuição Uniforme
area_cru = np.mean(halton_samples_y)
variancia = np.sum((halton_samples_y - area_cru)**2)/n
erro_padrao = np.sqrt(variancia)/np.sqrt(n)
end_time = time.time()
execution_time = end_time - start_time
total_time += execution_time
print(f"Área estimada: {area_cru}")
print(f"Variância do estimador: {variancia}")
print(f"Erro padrão do estimador: {erro_padrao:.10f}")
print(f"Largura da Distr.: {n}")
print()
n = round(((z_alfa * np.sqrt(variancia))/epsilon)**2)
print(f"Valor de n calculado: {n}")
print("Tempo de execução:", execution_time, "segundos \n")
print()

# ******************************************************************************* #
# ********************* MÉTODO DE MONTE CARLO 'HIT OR MISS' ********************* #
print("*** MÉTODO DE MONTE CARLO 'HIT OR MISS' ***")
# Aproximação através de Distribuição Uniforme
n = 100000
PontosDentro = 0
start_time = time.time()
hitmiss_samples_x = qmc.Halton(d=1).random(n)
hitmiss_samples_y = qmc.Halton(d=1).random(n)
for i in range(n):
    xi = hitmiss_samples_x[i]
    yi = hitmiss_samples_y[i]
    # Checa se o ponto sorteado está abaixo da curva
    if yi <= f(xi):
        PontosDentro += 1
area_hitmiss = PontosDentro/n
# Variancia do estimador
variancia = area_hitmiss*(1 - area_hitmiss)/n
erro_padrao = np.sqrt(variancia)/np.sqrt(n)
end_time = time.time()
execution_time = end_time - start_time
total_time += execution_time
print(f"Área estimada: {area_hitmiss}")
print(f"Variância do estimador: {variancia:.10f}")
print(f"Erro padrão do estimador: {erro_padrao:.10f}")
print(f"Largura da Distr.: {n}")
print()
n = round(((z_alfa * np.sqrt(variancia))/epsilon)**2)
print(f"Valor de n calculado: {n}")
print("Tempo de execução:", execution_time, "segundos")
print()

# ******************************************************************************************** #
# ********************* MÉTODO DE MONTE CARLO AMOSTRAGEM POR IMPORTÂNCIA ********************* #
print("*** MÉTODO DE MONTE CARLO AMOSTRAGEM POR IMPORTÂNCIA ***")
# Distribuição Uniforme
n = 100000
pesos = []
pdfx = []
start_time = time.time()
uniform_samples_x = qmc.Halton(d=1).random(n)
xi = uniform_samples_x
# Função Densidade de probabilidade da Distribuição Uniforme
pdfx = uniform.pdf(xi)
yi = f(xi)
# Calculo dos pesos, como o quociente entre os valores f(xi) da função pela FDP da distribuição uniforme
pesos = yi/pdfx
area_uniform = np.mean(np.array(pesos))
# Variancia do estimador
variancia = np.sum(np.array(pdfx)*(np.array(pesos) - area_uniform)**2)/n
erro_padrao = np.sqrt(variancia)/np.sqrt(n)
end_time = time.time()
execution_time = end_time - start_time
total_time += execution_time
print(f"Uniforme: Área estimada: {area_uniform}")
print(f"Uniforme: Variância do estimador: {variancia:.10f}")
print(f"Uniforme: Erro padrão do estimador: {erro_padrao:.10f}")
print(f"Uniforme: Quantidade de iterações: {n}")
print()
n = round(((z_alfa * np.sqrt(variancia))/epsilon)**2)
print(f"Valor de n calculado: {n}")
print("Tempo de execução:", execution_time, "segundos")
print()

# Distribuição Beta
# Implementação do método de Halton para gerar amostras quase aleatórias
pesos = []
pdfx = []
n = 100000
start_time = time.time()
beta_samples_x = qmc.Halton(d=1).random(n)
xi = beta_samples_x
# Transformação da variável quase-aleatória uniforme na distribuição Beta
xi = beta.ppf(xi, a = 1, b = 1)
# Obtenção da função densidade de probabilidade, para a distribuição Beta
pdfx = beta.pdf(xi, a = 1, b = 1)
yi = f(xi)
pesos = yi/pdfx
area_beta = np.mean(pesos)
# Variancia do estimador
variancia = np.sum(np.array(pdfx)*(np.array(pesos) - area_beta)**2)/n
erro_padrao = np.sqrt(variancia)/np.sqrt(n)
end_time = time.time()
execution_time = end_time - start_time
total_time += execution_time
print(f"Beta: Área estimada: {area_beta}")
print(f"Beta: Variância do estimador: {variancia:.10f}")
print(f"Beta: Erro padrão do estimador: {erro_padrao:.10f}")
print(f"Beta: Quantidade de iterações: {n}")
print()
n = round(((z_alfa * np.sqrt(variancia))/epsilon)**2)
print(f"Valor de n calculado: {n}")
print("Tempo de execução:", execution_time, "segundos")
print()


# Distribuição Gamma
# Normalização da distribuição Gamma a partir da função cumulativa nos pontos de interesse
cdf_lower = gamma.cdf(0, a=1, scale=2)
cdf_upper = gamma.cdf(1, a=1, scale=2)
const_normalizacao = 1 / (cdf_upper - cdf_lower)
# Função que gera valores quase aleatórios para a distribuição Gama
def generate_n_gamma_samples(num_samples):
    samples_no_intervalo = []
    while len(samples_no_intervalo) < num_samples:
        # Gerar amostras aleatórias pela distribuição Gama
        gamma_samples_x = qmc.Halton(d=1).random(1)
        gamma_samples_x = gamma.ppf(gamma_samples_x, a= 1, scale = 2)
        # Checa se a amostra está dentro do intervalo
        if 0 <= gamma_samples_x <= 1:
            samples_no_intervalo.append(gamma_samples_x)
    return np.array(samples_no_intervalo)

pesos = []
pdfx = []
n = 100000
start_time = time.time()
gamma_samples_x = generate_n_gamma_samples(n)
xi = gamma_samples_x
# Obtenção da função densidade de probabilidade, para a distribuição Gamma, e normalizada à 0 e 1
pdf_value = gamma.pdf(xi, a = 1, scale = 2)
pdfx = pdf_value * const_normalizacao
yi = f(xi)
pesos = yi/pdfx
area_gama = np.mean(pesos)
# Variancia do estimador
variancia = np.sum(np.array(pdfx)*(np.array(pesos) - area_gama)**2)/n
erro_padrao = np.sqrt(variancia)/np.sqrt(n)
end_time = time.time()
execution_time = end_time - start_time
total_time += execution_time
print(f"Gama: Área estimada: {area_gama}")
print(f"Gama: Variância do estimador: {variancia}")
print(f"Gama: Erro padrão do estimador: {erro_padrao:.10f}")
print(f"Gama: Quantidade de iterações: {n}")
print()
n = round(((z_alfa * np.sqrt(variancia))/epsilon)**2)
print(f"Valor de n calculado: {n}")
print("Tempo de execução:", execution_time, "segundos")
print()

# Distribuição Weibull
# Normalização da distribuição Weibull a partir da função cumulativa nos pontos de interesse
cdf_lower = weibull_min.cdf(0, c=1)
cdf_upper = weibull_min.cdf(1, c=1)
const_normalizacao = 1 / (cdf_upper - cdf_lower)
# Função que gera valores quase aleatórios para a distribuição Weibull
def generate_n_weibull_samples(num_samples):
    samples_no_intervalo = []
    while len(samples_no_intervalo) < num_samples:
        # Gerar amostras aleatórias pela distribuição Gama
        weibull_samples_x = qmc.Halton(d=1).random(1)
        weibull_samples_x = weibull_min.ppf(weibull_samples_x, c=1)
        # Checa se a amostra está dentro do intervalo
        if 0 <= weibull_samples_x <= 1:
            samples_no_intervalo.append(weibull_samples_x)
    return np.array(samples_no_intervalo)

pesos = []
pdfx = []
n = 100000
start_time = time.time()
weibull_samples_x = generate_n_weibull_samples(n)
xi = weibull_samples_x
# Obtenção da função densidade de probabilidade, para a distribuição Weibull, e normalizada à 0 e 1
pdf_value = weibull_min.pdf(xi, c=1)
pdfx = pdf_value * const_normalizacao
yi = f(xi)
pesos = yi/pdfx
area_weibull = np.mean(pesos)
# Variancia do estimador
variancia = np.sum(np.array(pdfx)*(np.array(pesos) - area_weibull)**2)/n
erro_padrao = np.sqrt(variancia)/np.sqrt(n)
end_time = time.time()
execution_time = end_time - start_time
total_time += execution_time
print(f"Weibull: Área estimada: {area_weibull}")
print(f"Weibull: Variância do estimador: {variancia:.10f}")
print(f"Weibull: Erro padrão do estimador: {erro_padrao:.10f}")
print(f"Weibull: Quantidade de iterações: {n}")
print()
n = round(((z_alfa * np.sqrt(variancia))/epsilon)**2)
print(f"Valor de n calculado: {n}")
print("Tempo de execução:", execution_time, "segundos")
print()

# *************************************************************************************** #
# ********************* MÉTODO DE MONTE CARLO VARIÁVEIS DE CONTROLE ********************* #
print("*** MÉTODO DE MONTE CARLO VARIÁVEIS DE CONTROLE ***")
# Definição dos limites de integração para a função f e φ
limInfer, limSuper= 0, 1
# Função polinominal φ que aproxima-se à curva da função f(x), encontrada a partir dos pontos (1, f(1)) e (0, f(0))
def phi(x):
    phix = ((f(1)-f(0))/(1 - 0))*x + 1
    #phix = 1 - 0.564263989*x + 0.02814*x**(2) + 0.04400*x**(3)- 0.01377*x**(4)
    return phix
# Função primitiva de φ
def phiPrim(x):
    phiPrim = x + ((f(1)-f(0))/(1 - 0))*x**(2)/2
    #phiPrim = x - 0.564263989*x**(2)/2 + 0.02814*x**(3)/3 + 0.04400*x**(4)/4 - 0.01377*x**(5)/5
    return phiPrim

termo = 0
fxn, phixn = [], []
n = 100000
start_time = time.time()
controlv_samples_x = qmc.Halton(d=1).random(n)
#xi = controlv_samples_x[i]
# Armezando os valores de f(xi) e g(xi) para o cálculo final da variância
fxn = f(controlv_samples_x)
phixn = phi(controlv_samples_x)
# Termo do somatório para calculo de gama chapeu
termo = np.sum(f(controlv_samples_x) - phi(controlv_samples_x) + (phiPrim(limSuper)-phiPrim(limInfer)))
area_control = termo/n
var_f = np.var(np.array(fxn))
var_phi = np.var(np.array(phixn))
correlacao = np.cov(fxn, phixn, rowvar=False)[0,1]
variancia = (1/n)*(var_f + var_phi - 2*correlacao*np.sqrt(var_f)*np.sqrt(var_phi))
erro_padrao = np.sqrt(variancia)/np.sqrt(n)
end_time = time.time()
execution_time = end_time - start_time
total_time += execution_time
print(f"Área estimada:: {area_control}")
print(f"Variância do estimador: {variancia:.10f}")
print(f"Erro padrão do estimador: {erro_padrao:.10f}")
print(f"Quantidade de iterações: {n}")
print()
n = round(((z_alfa * np.sqrt(variancia))/epsilon)**2)
print(f"Valor de n calculado: {n}")
print("Tempo de execução:", execution_time, "segundos")
print()
print("Tempo total de execução:", total_time, "segundos")

*** EP 3 INICIANDO ***

*** EXPERIMENTO PILOTO ***
Área Piloto 0.7357866720137144

*** MÉTODO DE MONTE CARLO CRU ***
Área estimada: 0.735785903718138
Variância do estimador: 0.021390949082884345
Erro padrão do estimador: 0.0004625035
Largura da Distr.: 100000

Valor de n calculado: 425083
Tempo de execução: 0.09066987037658691 segundos 


*** MÉTODO DE MONTE CARLO 'HIT OR MISS' ***
Área estimada: 0.53125
Variância do estimador: 0.0000024902
Erro padrão do estimador: 0.0000049902
Largura da Distr.: 100000

Valor de n calculado: 49
Tempo de execução: 0.761049747467041 segundos

*** MÉTODO DE MONTE CARLO AMOSTRAGEM POR IMPORTÂNCIA ***
Uniforme: Área estimada: 0.7357844594096536
Uniforme: Variância do estimador: 0.0213909138
Uniforme: Erro padrão do estimador: 0.0004625031
Uniforme: Quantidade de iterações: 100000

Valor de n calculado: 425082
Tempo de execução: 0.09574389457702637 segundos

Beta: Área estimada: 0.7357849510292592
Beta: Variância do estimador: 0.0213909343
Beta: Erro padrã